In [1]:
#OS
import os
from os import path
import logging

#Libraries
import cv2
import math
import pafy
import random
import numpy as np
import datetime as dt
from moviepy.editor import *
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

#Models
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
from keras.models import load_model

#Setting Randomness
seed_constant = 23
np.random.seed(seed_constant)
random.seed(seed_constant)
tf.random.set_seed(seed_constant)

In [2]:
#Downloading YouTube Video
def download_youtube_videos(youtube_video_url, output_directory):
    video = pafy.new(youtube_video_url)
    video_best = video.getbest()
    output_file_path = f'{output_directory}/{video.title}.mp4'
    video_best.download(filepath = output_file_path, quiet = True)
    return video.title

In [3]:
#Making Predictions using model
def predict_on_video(input_file, output_file_path, window_size):
    predicted_labels_probabilities_deque = deque(maxlen = window_size)
    video_reader = cv2.VideoCapture(input_file)
    original_video_width = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_video_height = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))
    video_writer = cv2.VideoWriter(output_file_path, cv2.VideoWriter_fourcc('M', 'P', '4', 'V'), 24, (original_video_width, original_video_height))

    while True: 
        status, frame = video_reader.read() 

        if not status:
            break

        resized_frame = cv2.resize(frame, (image_height, image_width))
        normalized_frame = resized_frame / 255
        predicted_labels_probabilities = model.predict(np.expand_dims(normalized_frame, axis = 0))[0]
        predicted_labels_probabilities_deque.append(predicted_labels_probabilities)

        if len(predicted_labels_probabilities_deque) == window_size:
            predicted_labels_probabilities_np = np.array(predicted_labels_probabilities_deque)
            predicted_labels_probabilities_averaged = predicted_labels_probabilities_np.mean(axis = 0)
            predicted_label = np.argmax(predicted_labels_probabilities_averaged)
            predicted_class_name = classes_list[predicted_label]
            cv2.putText(frame, predicted_class_name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        video_writer.write(frame)
        #cv2.imshow('Predicted Frames', frame)
        #key_pressed = cv2.waitKey(10)

        #if key_pressed == ord('q'):
        #    break

    cv2.destroyAllWindows()
    video_reader.release()
    video_writer.release()

In [4]:
#Reading the model
model = load_model('Activity Detection')

image_height, image_width = 64, 64
max_images_per_class = 8000

In [5]:
dataset_directory = "UCF-101"
classes_list = ['ApplyEyeMakeup','ApplyLipstick','BlowDryHair','BrushingTeeth','Typing', 'WritingOnBoard']

In [ ]:
#Flask
from flask import Flask, request, redirect, url_for, session, jsonify, send_from_directory
from werkzeug.utils import secure_filename

app = Flask(__name__)
app.secret_key = '123'

#Setting up upload
UPLOAD_FOLDER="uploads"
#ALLOWED_EXTENSIONS = set([])

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('Upload Server')

app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

#CROS
@app.after_request
def add_headers(response):
    response.headers.add('Content-Type', 'application/json')
    response.headers.add('Access-Control-Allow-Origin', '*')
    response.headers.add('Access-Control-Allow-Methods', 'PUT, GET, POST, DELETE, OPTIONS')
    response.headers.add('Access-Control-Allow-Headers', 'Content-Type,Authorization')
    response.headers.add('Access-Control-Expose-Headers', 'Content-Type,Content-Length,Authorization,X-Pagination')
    return response

@app.route('/fileUpload', methods=['POST'])
def fileUpload():
	if request.method == 'POST':
		target=path.join(UPLOAD_FOLDER)
		file = request.files['file']
		filename = secure_filename(file.filename)
		destination="/".join([target, filename])
		file.save(destination)
		session['uploadFilePath']=destination
		logger.info("Connection Established.")

		predict_on_video(destination, "output.mp4", 25)

	if path.exists("output.mp4"):
		print("Success.")
		return send_from_directory(".", "output.mp4", as_attachment=True)

	else:
		print("Failed.")
		status={"success":"false"}
		return jsonify(status)

@app.route('/linkUpload', methods=['POST'])
def linkUpload():
	if request.method == 'POST':
		multi_dict = request.form
		link = request.form['website']
		print("Downloading Video")
		video_name = download_youtube_videos(link, UPLOAD_FOLDER)
		print("Downloaded Video")
		filename = video_name+".mp4"
		destination="/".join([UPLOAD_FOLDER, filename])
		session['uploadFilePath']=destination

		print("Predicting Video")
		predict_on_video(destination, "output.mp4", 25)
		print("Predicted Video")
	
	if path.exists("output.mp4"):
		print("Success.")
		return send_from_directory(".", "output.mp4", as_attachment=True, mimetype="video/mp4")

	else:
		print("Failed.")
		status={"success":"false"}
		return jsonify(status)

app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)



Downloaded Video
Predicting Video


INFO:werkzeug:127.0.0.1 - - [29/Jan/2022 11:32:14] "POST /linkUpload HTTP/1.1" 200 -


Predicted Video
Success.

Downloaded Video
Predicting Video


INFO:werkzeug:127.0.0.1 - - [29/Jan/2022 11:34:53] "POST /linkUpload HTTP/1.1" 200 -


Predicted Video
Success.
